# 自然语言识别处理单元
## [拂晓工作室](https://github.com/Errrneist/Alchemist)
* 此程序可用于识别文字中的语言以及深度分析新闻概况

# 参考资料
* [1] [SFrame入门](https://apple.github.io/turicreate/docs/api/generated/turicreate.SFrame.html#turicreate.SFrame)
* [2] [从字符串中删除子字符串](https://stackoverflow.com/questions/31273642/better-way-to-remove-multiple-words-from-a-string)
* [3] [用Python分析《红楼梦》 - 楼宇](https://zhuanlan.zhihu.com/p/29209681)
* [4] [使用pyltp包进行中文分词实例](https://blog.csdn.net/sinat_26917383/article/details/77067515)
* [5] [pyltp技术文档](http://pyltp.readthedocs.io/zh_CN/latest/api.html#id15)
* [6] [pyltp介绍文档一](https://www.ltp-cloud.com/intro/#introduction )
* [7] [pyltp介绍文档二](http://ltp.readthedocs.io/zh_CN/latest/appendix.html#id5)
* [8] [pyltp深度训练模型](https://pan.baidu.com/share/link?shareid=1988562907&uk=2738088569#list/path=/)


# 导入库

In [52]:
# 导入基本库
import re
import os
import time
import random
import datetime

# 网络获取相关包
import pymysql
import urllib
from bs4 import BeautifulSoup

# 机器学习与大数据框架
import turicreate as tc
import csv

# 自然语言识别框架
import pyltp
from pyltp import SentenceSplitter
from pyltp import Segmentor
from pyltp import Postagger
LTP_DATA_DIR = '../../../LTP_data_v3.4.0/'  # ltp模型目录的路径
pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')  # 词性标注模型路径，模型名称为`pos.model`
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`

# 载入数据

In [3]:
# 载入数据
news = tc.SFrame('../DataSets/Eastmoney/News/China/CHINA20180702-1906.csv')

# 预览
news

Finished parsing file /Users/hongjunwu/Documents/GitHub/Alchemist/DataSets/Eastmoney/News/China/CHINA20180702-1906.csv

Parsing completed. Parsed 100 lines in 0.047978 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,int,list,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/hongjunwu/Documents/GitHub/Alchemist/DataSets/Eastmoney/News/China/CHINA20180702-1906.csv

Parsing completed. Parsed 509 lines in 0.044427 secs.

contents,date,day,month,related
小微企业融资难、融资贵问题，是一个世界性难题。甚至，外延更为丰富的普惠金融，在很多时候、在不 ...,20180630,30,6,"[保险, 陆家嘴, 银行]"
据国家统计局30日消息，2018年6月份，中国制造业采购经理指数(PMI)为51.5%，比上 ...,20180630,30,6,[]
针对今年前5个月全国规模以上工业企业利润数据与去年同期数据的差异问题，国家统计局工业司有关负 ...,20180629,29,6,[]
今日，内蒙古自治区住房资金管理中心网站发布《关于进一步规范调整住房公积金提取政策的通知》称， ...,20180702,2,7,[]
自2016年10月份国务院发布《关于市场化银行债权转股权的指导意见》以来，债转股一直存在“落 ...,20180702,2,7,[银行]
污水处理、垃圾处理、节水、节能环保……国家发展改革委2日对外发布《关于创新和完善促进绿色发展 ...,20180702,2,7,[]
29日，中国人民银行会同银保监会、证监会、国家发展改革委、财政部等联合召开全国深化小微企业金 ...,20180630,30,6,[银行]
货币政策委员会二季度例会释放信号——日前召开的中国人民银行货币政策委员会2018年第二季度例 ...,20180702,2,7,[银行]
近日，由中国旅游研究院主办，人民日报客户端旅游频道提供媒体支持的《中国入境旅游发展年度报告2 ...,20180702,2,7,[]
作为银行发起设立的债转股实施机构，金融资产投资公司是一类新型实施机构。从昨日开始，这类机构有 ...,20180630,30,6,[银行]


# 检查数据纯净度
* 测试用，检查清洗数据的效果

In [10]:
# 随机人工检查数据纯净度

def showContents(number_to_show, total_amount):
    counter = 0
    while counter < number_to_show:
        print(news['contents'][int(random.uniform(0, total_amount - 1))])
        print('')
        counter += 1
    
showContents(3, len(news['contents']))

新华社成都6月29日电(记者陈健)记者29日从四川省国土资源厅了解到，6月24日至今，四川迎来今年入汛后强度最大、范围最广、持续时间最长的一次强降雨天气过程，导致泥石流、滑坡等地质灾害频发。截至目前，四川组织地质灾害主动预防避让27987人，各地发生较大规模的地质灾害203起，未造成人员伤亡。据介绍，24日以来，四川向全省预警区发布地质灾害3级以上(含3级)地质灾害气象风险预警4次，向各隐患点防灾责任人及监测员发布预警短信4.4万余条。从抽查检查情况看，相关防灾责任人及地质灾害隐患点群测群防专职监测员上岗履职情况良好，信息畅通。在备受关注的九寨沟地震灾区，共发生地质灾害灾情和险情6处，当地及时提示游客主动绕避危险区域，抓好在建工程施工区域地质灾害防范管控，由于措施得当，无人员因灾伤亡。此外，四川还加强了重点区域现场一线技术力量，以部署在全省各地的197支专业应急队伍为支撑，统筹部署了540余名专业技术人员协助指导驻地政府及相关部门全力做好地质灾害防范。

近年来，云南省把节能降耗作为打好污染防治攻坚战、推动经济高质量发展的突破口和重要抓手。5年来，云南省全力推进节能降耗各项工作，实现节能量2945万吨标准煤，万元工业增加值能耗累计下降24%，实现节能量2580万吨标准煤，全省单位GDP能耗累计下降23%。近5年来，云南省各级相关部门，贯彻落实党中央、国务院、省委、省政府决策部署，采取有效措施，全力推进节能降耗各项工作，全省节能降耗取得显著成效。云南省节能法规、政策、标准体系进一步完善，节能科技支撑作用和节能市场化水平进一步提高，节能监督管理体制机制进一步完善，主要耗能产品单位能耗持续下降。省级财政安排节能专项资金2.72亿元，支持250个项目建设，实现节能量303万吨标准煤。各州市大力推进燃煤锅炉节能环保综合改造，为保卫蓝天作出了重要贡献。全省建筑节能全面推进，城镇新建建筑设计阶段执行建筑节能设计标准比例达100%。绿色低碳交通基础设施建设取得新成绩，武易高速等多条高速公路被交通运输部列为绿色公路建设典型示范工程、生态文明示范路。公共机构发挥节能表率作用，为建设生态文明排头兵提供了重要支撑，为绿色发展注入了新活力、新动力。

6月29日从生态环境部例行发布会获悉，20182019年蓝天保卫战重点区域强化督查，从去年的京津冀及周边地区，扩展到了汾渭平原和长三角地区，从

# 自然语言识别

# 第一步 分句分词

# 分句函数
* 将句子分开并创建名为'content_split_sents'的列

In [44]:
# 分句函数
def splitSents(contents):
    return SentenceSplitter.split(contents)

# 执行分句函数
news['content_split_sents'] = news['contents'].apply(splitSents)

# 预览
news

contents,date,day,month,related
小微企业融资难、融资贵问题，是一个世界性难题。甚至，外延更为丰富的普惠金融，在很多时候、在不 ...,20180630,30,6,"[保险, 陆家嘴, 银行]"
据国家统计局30日消息，2018年6月份，中国制造业采购经理指数(PMI)为51.5%，比上 ...,20180630,30,6,[]
针对今年前5个月全国规模以上工业企业利润数据与去年同期数据的差异问题，国家统计局工业司有关负 ...,20180629,29,6,[]
今日，内蒙古自治区住房资金管理中心网站发布《关于进一步规范调整住房公积金提取政策的通知》称， ...,20180702,2,7,[]
自2016年10月份国务院发布《关于市场化银行债权转股权的指导意见》以来，债转股一直存在“落 ...,20180702,2,7,[银行]
污水处理、垃圾处理、节水、节能环保……国家发展改革委2日对外发布《关于创新和完善促进绿色发展 ...,20180702,2,7,[]
29日，中国人民银行会同银保监会、证监会、国家发展改革委、财政部等联合召开全国深化小微企业金 ...,20180630,30,6,[银行]
货币政策委员会二季度例会释放信号——日前召开的中国人民银行货币政策委员会2018年第二季度例 ...,20180702,2,7,[银行]
近日，由中国旅游研究院主办，人民日报客户端旅游频道提供媒体支持的《中国入境旅游发展年度报告2 ...,20180702,2,7,[]
作为银行发起设立的债转股实施机构，金融资产投资公司是一类新型实施机构。从昨日开始，这类机构有 ...,20180630,30,6,[银行]


# 分词函数
* 将词分开 并创建名为'contents_split_words'的列

In [48]:
# 分词函数
def splitWords(contents):
    segmentor = Segmentor()  # 初始化实例
    segmentor.load(cws_model_path)  # 加载模型
    words = segmentor.segment(contents)  # 分词
    segmentor.release()  # 释放模型
    return words

# 执行分词函数
news['content_split_words'] = news['contents'].apply(splitWords)

# 预览
news

contents,date,day,month,related
小微企业融资难、融资贵问题，是一个世界性难题。甚至，外延更为丰富的普惠金融，在很多时候、在不 ...,20180630,30,6,"[保险, 陆家嘴, 银行]"
据国家统计局30日消息，2018年6月份，中国制造业采购经理指数(PMI)为51.5%，比上 ...,20180630,30,6,[]
针对今年前5个月全国规模以上工业企业利润数据与去年同期数据的差异问题，国家统计局工业司有关负 ...,20180629,29,6,[]
今日，内蒙古自治区住房资金管理中心网站发布《关于进一步规范调整住房公积金提取政策的通知》称， ...,20180702,2,7,[]
自2016年10月份国务院发布《关于市场化银行债权转股权的指导意见》以来，债转股一直存在“落 ...,20180702,2,7,[银行]
污水处理、垃圾处理、节水、节能环保……国家发展改革委2日对外发布《关于创新和完善促进绿色发展 ...,20180702,2,7,[]
29日，中国人民银行会同银保监会、证监会、国家发展改革委、财政部等联合召开全国深化小微企业金 ...,20180630,30,6,[银行]
货币政策委员会二季度例会释放信号——日前召开的中国人民银行货币政策委员会2018年第二季度例 ...,20180702,2,7,[银行]
近日，由中国旅游研究院主办，人民日报客户端旅游频道提供媒体支持的《中国入境旅游发展年度报告2 ...,20180702,2,7,[]
作为银行发起设立的债转股实施机构，金融资产投资公司是一类新型实施机构。从昨日开始，这类机构有 ...,20180630,30,6,[银行]


# 第二步 词性标注

In [63]:
def tagWords(contents_split):
    postagger = Postagger() # 初始化实例
    postagger.load(pos_model_path)  # 加载模型

    words = contents_split  # 分词结果
    postags = postagger.postag(words)  # 词性标注

    postagger.release()  # 释放模型
    return postags

In [ ]:
# 执行分词函数
news['content_tag_words'] = news['content_split_words'].apply(tagWords)

# 预览
news

# 第三步 命名实体识别

# 第四步 依存句法分析

# 第五步 语义角色分析